## Library
---

In [2]:
import os
import time
import numpy as np


## Algorithm
---

In [3]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [4]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
'''
bond_energy = 2.07 
temperature = 700
e0 = 0.1
num_nearest_atoms = 12

# site energy, e_(bond number)
e_site = np.zeros(num_nearest_atoms)
for i in range(num_nearest_atoms):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_nearest_atoms, num_nearest_atoms))
for i in range(num_nearest_atoms):
    for j in range(num_nearest_atoms):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_nearest_atoms, num_nearest_atoms))
for i in range(num_nearest_atoms):
    for j in range(num_nearest_atoms):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [5]:
diffusion_rate

array([[1.90559975e+12, 1.05973894e+13, 5.89340238e+13, 3.27742904e+14,
        1.82263833e+15, 1.01360257e+16, 5.63682960e+16, 3.13474422e+17,
        1.74328869e+18, 9.69474780e+18, 5.39142687e+19, 2.99827126e+20],
       [1.21113472e+04, 1.90559975e+12, 1.05973894e+13, 5.89340238e+13,
        3.27742904e+14, 1.82263833e+15, 1.01360257e+16, 5.63682960e+16,
        3.13474422e+17, 1.74328869e+18, 9.69474780e+18, 5.39142687e+19],
       [7.69756242e-05, 1.21113472e+04, 1.90559975e+12, 1.05973894e+13,
        5.89340238e+13, 3.27742904e+14, 1.82263833e+15, 1.01360257e+16,
        5.63682960e+16, 3.13474422e+17, 1.74328869e+18, 9.69474780e+18],
       [4.89231019e-13, 7.69756242e-05, 1.21113472e+04, 1.90559975e+12,
        1.05973894e+13, 5.89340238e+13, 3.27742904e+14, 1.82263833e+15,
        1.01360257e+16, 5.63682960e+16, 3.13474422e+17, 1.74328869e+18],
       [3.10938680e-21, 4.89231019e-13, 7.69756242e-05, 1.21113472e+04,
        1.90559975e+12, 1.05973894e+13, 5.89340238e+13, 3.27

In [6]:
# fcc and bcc using numpy!
def create_fcc_lattice(height, width, depth):
    # Create a 3D matrix filled with zeros
    lattice = np.zeros((height, width, depth), dtype=int)

    # Fill the matrix with ones at the positions of the atoms in the FCC lattice
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                if (x % 2 == 0 and y % 2 == 0 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 1 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 0 and z % 2 == 1) or \
                   (x % 2 == 0 and y % 2 == 1 and z % 2 == 1):
                    lattice[z, x, y] = 1
    
    # add vaccum layers
    vaccum = np.zeros((1, width, depth))
    lattice = np.concatenate((vaccum, lattice, vaccum), axis=0)

    return lattice

def create_bcc_lattice(height, width, depth):
    # Create a 3D matrix filled with zeros
    lattice = np.zeros((height, width, depth))

    # Fill the matrix with ones at the positions of the atoms in the BCC lattice
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                if (x % 2 == 0 and y % 2 == 0 and z % 2 == 0) or \
                   (x % 2 == 1 and y % 2 == 1 and z % 2 == 1):
                    lattice[z, x, y] = 1

    # add vaccum layers
    vaccum = np.zeros((1, width, depth))
    lattice = np.concatenate((vaccum, lattice, vaccum), axis=0)

    return lattice

def create_simple_cubic_lattice(height, width, depth):
    lattice = np.ones((height+2, width, depth), dtype=int)
    lattice[0, :, :] = 0
    lattice[-1, :, :] = 0

    return lattice


In [7]:
# lattice to xyz
def save_matrix_as_xyz(matrix, atom, output_filename):
    with open(output_filename, 'a') as output_file:
        num_atoms = int(np.sum(matrix))  # Calculate the number of Cu atoms (1s)

        # Write the number of atoms as the first line in the XYZ file
        output_file.write(f"{num_atoms}\n\n")

        # Iterate through the matrix to write the atom positions
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                for k in range(matrix.shape[2]):
                    if matrix[i, j, k] == 1:  # Cu atom
                        output_file.write(f"{atom} {i} {j} {k}\n")

In [8]:
test_fcc = create_fcc_lattice(3, 10, 10)
test_bcc = create_bcc_lattice(10, 10, 10)

In [9]:
'''
find nearest atoms around the site
'''
def get_atoms_around_site(lattice, z, x, y, lattice_type='fcc'):
    height, width, depth = lattice.shape
    current_pos = np.array([z, x, y])
    if lattice_type == 'fcc':
        nearest_atoms = np.array([[1, 1, 0], [-1, 1, 0], [1, -1, 0], [-1, -1, 0],
                                [1, 0, 1], [-1, 0, 1], [1, 0, -1], [-1, 0, -1],
                                [0, 1, 1], [0, -1, 1], [0, 1, -1], [0, -1, -1]])
    elif lattice_type == 'bcc':
        nearest_atoms = np.array([[1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1],
                              [-1, -1, -1], [-1, -1, 1], [-1, 1, -1], [1, -1, -1]])
    nearest_atoms_pos = nearest_atoms + current_pos
    atoms_list = []
    vacancy_list = []

    for pos in nearest_atoms_pos:
        if 0 <= pos[0] <= height-1:
            if lattice[pos[0], (pos[1]+width)%width, (pos[2]+depth)%depth] == 1:
                atoms_list.append([pos[0], (pos[1]+width)%width, (pos[2]+depth)%depth])
            else :
                vacancy_list.append([pos[0], (pos[1]+width)%width, (pos[2]+depth)%depth])
    return atoms_list, vacancy_list


In [10]:
# test
get_atoms_around_site(test_fcc, 1, 1, 1, 'fcc')

([[2, 2, 1],
  [2, 0, 1],
  [2, 1, 2],
  [2, 1, 0],
  [1, 2, 2],
  [1, 0, 2],
  [1, 2, 0],
  [1, 0, 0]],
 [[0, 2, 1], [0, 0, 1], [0, 1, 2], [0, 1, 0]])

In [11]:
# find candidate
def find_candidate(lattice, lattice_type='fcc'):
    global diffusion_rate
    height, width, depth = lattice.shape
    candidate_table = []
    diffusion_table = []
    motion_table = []
    for z in range(height):
        for x in range(width):
            for y in range(depth):
                # find atom
                if lattice[z, x, y] == 1:
                    atoms_list, vacancy_list = get_atoms_around_site(lattice, z, x, y, lattice_type)
                    for vacancy_pos in vacancy_list:
                        candidate_table.append((z, x, y))
                        motion_table.append(vacancy_pos)
                        num_of_neighbors_of_current = len(atoms_list)
                        next_pos_atoms_list, _ = get_atoms_around_site(lattice, *vacancy_pos, lattice_type)
                        # print(num_of_neighbors_of_current, len(next_pos_atoms_list)-1)
                        diffusion_table.append(diffusion_rate[num_of_neighbors_of_current, len(next_pos_atoms_list)-1])
                        
    
    return candidate_table, diffusion_table, motion_table


In [12]:
cand, dif, mo = find_candidate(test_fcc, 'fcc')

In [13]:
len(cand)

400

In [14]:
# KMC function
def diffuse_one_step(lattice, lattice_type='fcc', print_out=False):
    global time_elapsed
    width, depth = lattice.shape[1], lattice.shape[2]

    cand, dif, motion = find_candidate(lattice, lattice_type)
    dif = np.array(dif)

    total_dif = np.sum(dif)
    
    # pick 1
    u = np.random.uniform(low=1e-6, high=1)
    u_time = np.random.uniform(low=1e-6, high=1)
    cum_dif = np.cumsum(dif)

    chosen_idx = np.argwhere(u*total_dif < cum_dif)[0][0]

    # print information
    if print_out:
        print(f'total_diff : {total_dif}')
        print(f'chosen : {cand[chosen_idx]} atom')   
    # print(f'motion : {motion[chosen_idx]}')
    # print(cand[chosen_idx])
    # print(motion[chosen_idx])

    # change the lattice
    z_from, x_from, y_from = cand[chosen_idx][0], cand[chosen_idx][1], cand[chosen_idx][2]
    z_to, x_to, y_to = motion[chosen_idx][0], motion[chosen_idx][1], motion[chosen_idx][2]

    lattice[z_from, x_from, y_from] = 0
    lattice[z_to, x_to, y_to] = 1
    
    # time update
    delta_t = -np.log(u_time)/total_dif
    # print(f'delta_t : {delta_t}')
    time_elapsed += delta_t

## Parameter setting
---

In [15]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
fcc -> 12 nearest neighbors
'''
bond_energy = 2.07 
temperature = 500
e0 = 0.1
num_closest_neighbors = 12

# site energy, e_(bond number)
e_site = np.zeros(num_closest_neighbors)
for i in range(num_closest_neighbors):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [16]:
# value check
# print(e_site)
# print('-----------------')
# # print(e_a)
# print('-----------------')
# print(diffusion_rate)

## Simulation
---

In [19]:
# parameter tuning
time_elapsed = 0
save_file_name = 'ovito/np_fcc_simul_test2.xyz'
steps = 1000
width = 100
depth = 100
height = 10
atom_type = 'Cu'

# create lattice
lattice = create_fcc_lattice(height, width, depth)
save_matrix_as_xyz(lattice, atom_type, save_file_name)
atom_num = lattice.sum()
real_time_start = time.time()

# simulation start
for i in range(1, steps+1):
    diffuse_one_step(lattice, 'fcc')
    if i%100 == 0:
        print(f'{i}th step is done')
        print(f'time elapsed : {time_elapsed}')
        save_matrix_as_xyz(lattice, atom_type, save_file_name)

    if lattice.sum() != atom_num:
        print('error!')
        break

100th step is done
time elapsed : 2.5366729913440407e+42
200th step is done
time elapsed : 6.43560477005018e+42


KeyboardInterrupt: 

## Test area
---

In [116]:
diffuse_one_step(test_fcc, 'fcc', print_out=True)

total_diff : 1.6452417215505484e-43
chosen : (1, 0, 4) atom


In [118]:
test_fcc[0]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])